<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: black ; color : skyblue; text-align: center; border-radius: 100px 100px;padding:10px">Tabular Playground Series-July2021</h1>
<br>

In this competition you are predicting the values of air pollution measurements over time, based on basic weather information (temperature and humidity) and the input values of 5 sensors.

The three target values to you to predict are: `target_carbon_monoxide`, `target_benzene`, and `target_nitrogen_oxides`

<br>
<h1 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: black ; color : skyblue; text-align: center; border-radius: 100px 100px;padding:10px">About Data</h1>
<br>

**train.csv** - the training data, including the weather data, sensor data, and values for the 3 targets<br>
**test.csv** - the same format as train.csv, but without the target value; your task is to predict the value for each of these targets<br>
**sample_submission.csv** - a sample submission file in the correct format.

<br>
<h1 style = "font-size:30px; font-family:Garamond ; font-weight : normal; background-color: black ; color : skyblue; text-align: center; border-radius: 100px 100px;padding:10px">Import library</h1>
<br>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_squared_log_error,r2_score
import catboost
from catboost import CatBoostRegressor
from catboost import Pool, cv
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from scipy.special import inv_boxcox
import plotly.express as ex
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
le = LabelEncoder()

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
train.head()

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Distribution of target variables</span></h1><br>

In [ ]:
ex.histogram(train['target_carbon_monoxide'])

In [ ]:
ex.histogram(train['target_benzene'])

In [ ]:
ex.histogram(train['target_nitrogen_oxides'])

All the 3 predicted variables seems to be right skewed a bit

In [ ]:
col=['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']

In [ ]:
X = train
Y1 = X['target_carbon_monoxide'].values
Y2 = X['target_benzene'].values
Y3 = X['target_nitrogen_oxides'].values
X = X.drop(['target_carbon_monoxide','target_benzene','target_nitrogen_oxides','date_time'], axis = 1)
X = X[col]

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Fit on target 1</span></h1><br>

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split (X, Y1, test_size = 0.20, random_state=42)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
all_regressors = []
all_regressors.append(('LR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
all_regressors.append(('LASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
all_regressors.append(('EN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
all_regressors.append(('Ridge', Pipeline([('Scaler', StandardScaler()),('RIDGE', Ridge())])))
all_regressors.append(('BYR', Pipeline([('Scaler', StandardScaler()),('BYR', BayesianRidge())])))
all_regressors.append(('KNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
all_regressors.append(('CART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
all_regressors.append(('SVM', Pipeline([('Scaler', StandardScaler()),('SVM', svm.SVR())])))
all_regressors.append(('XGB', Pipeline([('Scaler', StandardScaler()),('XGB', xgb.XGBRegressor())])))

train_error = []
test_error = []
train_error2 = []
test_error2 = []
for name, model in all_regressors:
    kfold = KFold(n_splits=5)
    cv_results_train = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='neg_mean_squared_log_error')
    cv_results_test = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='neg_mean_squared_log_error')
    cv_results_train2 = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='r2')
    cv_results_test2 = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='r2')
    #cv_results_train3 = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='neq_root_mean_squared_error')
    #cv_results_test3 = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='neq_root_mean_squared_error')
    train_error.append(cv_results_train.mean())
    test_error.append(cv_results_test.mean())
    train_error2.append(cv_results_train2.mean())
    test_error2.append(cv_results_test2.mean())

In [ ]:
col={'Train Error NegMsle':train_error,'Test Error NegMsle':test_error, 'Train Error R^2':train_error2,'Test Error R^2':test_error2}
models=['Linear Regression', 'Lasso Regression', 'ElasticNet', 'Ridge Regression', 'Bayesian Regression','KNN','Decision Tree','SVM', 'XGBoost']
rslt1=pd.DataFrame(data=col,index=models)
rslt1

From the above result on train set we can see Negmsle is low for model=SVM on test set having 0.022291

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Predict on test-Target1</span></h1><br>

In [ ]:
col=['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']

In [ ]:
sc=StandardScaler()
train_sc=sc.fit_transform(X)
test_sc=sc.transform(test[col])
model_svm=svm.SVR()

In [ ]:
model_svm.fit(train_sc,Y1)

In [ ]:
test_target1=model_svm.predict(test_sc)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Fit on target 2</span></h1><br>

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split (X, Y2, test_size = 0.20, random_state=42)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
all_regressors = []
all_regressors.append(('LR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
all_regressors.append(('LASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
all_regressors.append(('EN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
all_regressors.append(('Ridge', Pipeline([('Scaler', StandardScaler()),('RIDGE', Ridge())])))
all_regressors.append(('BYR', Pipeline([('Scaler', StandardScaler()),('BYR', BayesianRidge())])))
all_regressors.append(('KNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
all_regressors.append(('CART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
all_regressors.append(('SVM', Pipeline([('Scaler', StandardScaler()),('SVM', svm.SVR())])))
all_regressors.append(('XGB', Pipeline([('Scaler', StandardScaler()),('XGB', xgb.XGBRegressor())])))

train_error = []
test_error = []
train_error2 = []
test_error2 = []
for name, model in all_regressors:
    kfold = KFold(n_splits=5)
    cv_results_train = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='neg_mean_squared_log_error')
    cv_results_test = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='neg_mean_squared_log_error')
    cv_results_train2 = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='r2')
    cv_results_test2 = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='r2')
    #cv_results_train3 = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='neq_root_mean_squared_error')
    #cv_results_test3 = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='neq_root_mean_squared_error')
    train_error.append(cv_results_train.mean())
    test_error.append(cv_results_test.mean())
    train_error2.append(cv_results_train2.mean())
    test_error2.append(cv_results_test2.mean())

In [ ]:
col={'Train Error NegMsle':train_error,'Test Error NegMsle':test_error, 'Train Error R^2':train_error2,'Test Error R^2':test_error2}
models=['Linear Regression', 'Lasso Regression', 'ElasticNet', 'Ridge Regression', 'Bayesian Regression','KNN','Decision Tree','SVM', 'XGBoost']
rslt2=pd.DataFrame(data=col,index=models)
rslt2

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Predict on test-Target2</span></h1><br>

In [ ]:
col=['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']

In [ ]:
sc=StandardScaler()
train_sc=sc.fit_transform(X)
test_sc=sc.transform(test[col])
model_dt=DecisionTreeRegressor()

In [ ]:
model_dt.fit(train_sc,Y2)

In [ ]:
test_target2=model_dt.predict(test_sc)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Fit on target 3</span></h1><br>

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split (X, Y3, test_size = 0.20, random_state=42)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
all_regressors = []
all_regressors.append(('LR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
all_regressors.append(('LASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
all_regressors.append(('EN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
all_regressors.append(('Ridge', Pipeline([('Scaler', StandardScaler()),('RIDGE', Ridge())])))
all_regressors.append(('BYR', Pipeline([('Scaler', StandardScaler()),('BYR', BayesianRidge())])))
all_regressors.append(('KNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
all_regressors.append(('CART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
all_regressors.append(('SVM', Pipeline([('Scaler', StandardScaler()),('SVM', svm.SVR())])))
all_regressors.append(('XGB', Pipeline([('Scaler', StandardScaler()),('XGB', xgb.XGBRegressor())])))

train_error = []
test_error = []
train_error2 = []
test_error2 = []
for name, model in all_regressors:
    kfold = KFold(n_splits=5)
    cv_results_train = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='neg_mean_squared_log_error')
    cv_results_test = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='neg_mean_squared_log_error')
    cv_results_train2 = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='r2')
    cv_results_test2 = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='r2')
    #cv_results_train3 = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='neq_root_mean_squared_error')
    #cv_results_test3 = cross_val_score(model, X_test, Y_test, cv=kfold, scoring='neq_root_mean_squared_error')
    train_error.append(cv_results_train.mean())
    test_error.append(cv_results_test.mean())
    train_error2.append(cv_results_train2.mean())
    test_error2.append(cv_results_test2.mean())

In [ ]:
col={'Train Error NegMsle':train_error,'Test Error NegMsle':test_error, 'Train Error R^2':train_error2,'Test Error R^2':test_error2}
models=['Linear Regression', 'Lasso Regression', 'ElasticNet', 'Ridge Regression', 'Bayesian Regression','KNN','Decision Tree','SVM', 'XGBoost']
rslt3=pd.DataFrame(data=col,index=models)
rslt3

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Predict on test-Target3</span></h1><br>

In [ ]:
col=['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']

In [ ]:
sc=StandardScaler()
train_sc=sc.fit_transform(X)
test_sc=sc.transform(test[col])
model_xgb=xgb.XGBRegressor()

In [ ]:
model_xgb.fit(train_sc,Y3)

In [ ]:
test_target3=model_xgb.predict(test_sc)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Appending Results to Test Set</span></h1><br>

In [ ]:
sample_submission['target_carbon_monoxide']=test_target1
sample_submission['target_benzene']=test_target2
sample_submission['target_nitrogen_oxides']=test_target3

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.shape

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv',index=False)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Light GBM</span></h1><br>

In [ ]:
train.head()

In [ ]:
train['date_time_1'] = pd.to_datetime(train['date_time'])
train['month'] = train['date_time_1'].dt.month
train['dayofweek'] = train['date_time_1'].dt.dayofweek
train['hour'] = train['date_time_1'].dt.hour
test['date_time_1'] = pd.to_datetime(test['date_time'])
test['month'] = test['date_time_1'].dt.month
test['dayofweek'] = test['date_time_1'].dt.dayofweek
test['hour'] = test['date_time_1'].dt.hour

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">EDA- Across Month</span></h1><br>

In [ ]:
Y = 'target_carbon_monoxide'
X = 'month'
uniqueX = train[X].unique()
uniqueX = np.sort(uniqueX)

import plotly.graph_objects as go
fig = go.Figure()

for i in uniqueX:
    fig.add_trace(go.Violin(x=train[X][train[X] == i],
                            y=train[Y][train[X] == i],
                            name=str(i),
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title={'text':"Violin plots for target_carbon_monoxide hue on Month",
                         'xanchor': 'center',
                         'yanchor': 'top',
                         'x':0.5,'y':0.97},
                      template='plotly_dark',
                      legend=dict(yanchor="bottom",
                                  y=0.5,
                                  xanchor="center",
                                  x=1.2),
                      barmode='overlay',)

fig.show()

In [ ]:
Y = 'target_benzene'
X = 'month'
uniqueX = train[X].unique()
uniqueX = np.sort(uniqueX)

import plotly.graph_objects as go
fig = go.Figure()

for i in uniqueX:
    fig.add_trace(go.Violin(x=train[X][train[X] == i],
                            y=train[Y][train[X] == i],
                            name=str(i),
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title={'text':"Violin plots for target_benzene hue on Month",
                         'xanchor': 'center',
                         'yanchor': 'top',
                         'x':0.5,'y':0.97},
                      template='plotly_dark',
                      legend=dict(yanchor="bottom",
                                  y=0.5,
                                  xanchor="center",
                                  x=1.2),
                      barmode='overlay',)

fig.show()

In [ ]:
Y = 'target_nitrogen_oxides'
X = 'month'
uniqueX = train[X].unique()
uniqueX = np.sort(uniqueX)

import plotly.graph_objects as go
fig = go.Figure()

for i in uniqueX:
    fig.add_trace(go.Violin(x=train[X][train[X] == i],
                            y=train[Y][train[X] == i],
                            name=str(i),
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title={'text':"Violin plots for target_nitrogen_oxides hue on Month",
                         'xanchor': 'center',
                         'yanchor': 'top',
                         'x':0.5,'y':0.97},
                      template='plotly_dark',
                      legend=dict(yanchor="bottom",
                                  y=0.5,
                                  xanchor="center",
                                  x=1.2),
                      barmode='overlay',)

fig.show()

We can see some seasonal variation across the months on all the 3 target variables. As the median across the month is varying.

In [ ]:
Y = 'target_carbon_monoxide'
X = 'month'
uniqueX = train[X].unique()
uniqueX = np.sort(uniqueX)

import plotly.graph_objects as go
fig = go.Figure()

for i in uniqueX:
    fig.add_trace(go.Violin(x=train[X][train[X] == i],
                            y=train[Y][train[X] == i],
                            name=str(i),
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title={'text':"Violin plots for target_carbon_monoxide hue on Month",
                         'xanchor': 'center',
                         'yanchor': 'top',
                         'x':0.5,'y':0.97},
                      template='plotly_dark',
                      legend=dict(yanchor="bottom",
                                  y=0.5,
                                  xanchor="center",
                                  x=1.2),
                      barmode='overlay',)

fig.show()

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">EDA- Across Week</span></h1><br>

In [ ]:
Y = 'target_benzene'
X = 'dayofweek'
uniqueX = train[X].unique()
uniqueX = np.sort(uniqueX)

import plotly.graph_objects as go
fig = go.Figure()

for i in uniqueX:
    fig.add_trace(go.Violin(x=train[X][train[X] == i],
                            y=train[Y][train[X] == i],
                            name=str(i),
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title={'text':"Violin plots for target_carbon_monoxide hue on Week Day",
                         'xanchor': 'center',
                         'yanchor': 'top',
                         'x':0.5,'y':0.97},
                      template='plotly_dark',
                      legend=dict(yanchor="bottom",
                                  y=0.5,
                                  xanchor="center",
                                  x=1.2),
                      barmode='overlay',)

fig.show()

In [ ]:
Y = 'target_nitrogen_oxides'
X = 'dayofweek'
uniqueX = train[X].unique()
uniqueX = np.sort(uniqueX)

import plotly.graph_objects as go
fig = go.Figure()

for i in uniqueX:
    fig.add_trace(go.Violin(x=train[X][train[X] == i],
                            y=train[Y][train[X] == i],
                            name=str(i),
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title={'text':"Violin plots for target_nitrogen_oxides hue on Week Day",
                         'xanchor': 'center',
                         'yanchor': 'top',
                         'x':0.5,'y':0.97},
                      template='plotly_dark',
                      legend=dict(yanchor="bottom",
                                  y=0.5,
                                  xanchor="center",
                                  x=1.2),
                      barmode='overlay',)

fig.show()

In [ ]:
Y = 'target_carbon_monoxide'
X = 'dayofweek'
uniqueX = train[X].unique()
uniqueX = np.sort(uniqueX)

import plotly.graph_objects as go
fig = go.Figure()

for i in uniqueX:
    fig.add_trace(go.Violin(x=train[X][train[X] == i],
                            y=train[Y][train[X] == i],
                            name=str(i),
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title={'text':"Violin plots for target_carbon_monoxide hue on Week Day",
                         'xanchor': 'center',
                         'yanchor': 'top',
                         'x':0.5,'y':0.97},
                      template='plotly_dark',
                      legend=dict(yanchor="bottom",
                                  y=0.5,
                                  xanchor="center",
                                  x=1.2),
                      barmode='overlay',)

fig.show()

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Data Processing to Add seasonal Variables</span></h1><br>

In [ ]:
# Seasonal factors
ary =[]
for i in range(len(train["month"])):
    if(train["month"][i] == 1 or train["month"][i] == 2 or train["month"][i] == 12):
        ary.append(1)
    else:
        ary.append(0)
train["is_winter"] = ary

ary =[]
for i in range(len(train["month"])):
    if(train["month"][i] == 3 or train["month"][i] == 4 or train["month"][i] == 5):
        ary.append(1)
    else:
        ary.append(0)
train["is_spring"] = ary

ary =[]
for i in range(len(train["month"])):
    if(train["month"][i] == 6 or train["month"][i] == 7 or train["month"][i] == 8):
        ary.append(1)
    else:
        ary.append(0)
train["is_summer"] = ary

ary =[]
for i in range(len(train["month"])):
    if(train["month"][i] == 9 or train["month"][i] == 10 or train["month"][i] == 11):
        ary.append(1)
    else:
        ary.append(0)
train["is_autumn"] = ary

ary =[]
for i in range(len(test["month"])):
    if(test["month"][i] == 1 or test["month"][i] == 2 or test["month"][i] == 12):
        ary.append(1)
    else:
        ary.append(0)
test["is_winter"] = ary

ary =[]
for i in range(len(test["month"])):
    if(test["month"][i] == 3 or test["month"][i] == 4 or test["month"][i] == 5):
        ary.append(1)
    else:
        ary.append(0)
test["is_spring"] = ary

ary =[]
for i in range(len(test["month"])):
    if(test["month"][i] == 6 or test["month"][i] == 7 or test["month"][i] == 8):
        ary.append(1)
    else:
        ary.append(0)
test["is_summer"] = ary

ary =[]
for i in range(len(test["month"])):
    if(test["month"][i] == 9 or test["month"][i] == 10 or test["month"][i] == 11):
        ary.append(1)
    else:
        ary.append(0)
test["is_autumn"] = ary

# Week factors
train["is_weekend"] = (train["date_time_1"].dt.dayofweek >= 5).astype("int")
test["is_weekend"] = (test["date_time_1"].dt.dayofweek >= 5).astype("int")

# Time zone factors
ary =[]
for i in range(len(train["date_time_1"])):
    if (train["hour"][i] >= 8 and train["hour"][i] <= 9):
        ary.append(1)
    else:
        ary.append(0)
train["is_commute_m"] = ary

ary =[]
for i in range(len(train["date_time_1"])):
    if (train["hour"][i] >= 18 and train["hour"][i] <= 20):
        ary.append(1)
    else:
        ary.append(0)
train["is_commute_e"] = ary

ary =[]
for i in range(len(train["date_time_1"])):
    if (train["hour"][i] >= 10 and train["hour"][i] <= 16):
        ary.append(1)
    else:
        ary.append(0)
train["is_work"] = ary

ary =[]
for i in range(len(train["date_time_1"])):
    if (train["hour"][i] >= 23 or train["hour"][i] <= 5):
        ary.append(1)
    else:
        ary.append(0)
train["is_night"] = ary

ary =[]
for i in range(len(test["date_time_1"])):
    if (test["hour"][i] >= 8 and test["hour"][i] <= 9):
        ary.append(1)
    else:
        ary.append(0)
test["is_commute_m"] = ary

ary =[]
for i in range(len(test["date_time_1"])):
    if (test["hour"][i] >= 18 and test["hour"][i] <= 20):
        ary.append(1)
    else:
        ary.append(0)
test["is_commute_e"] = ary

ary =[]
for i in range(len(test["date_time_1"])):
    if (test["hour"][i] >= 10 and test["hour"][i] <= 16):
        ary.append(1)
    else:
        ary.append(0)
test["is_work"] = ary

ary =[]
for i in range(len(test["date_time_1"])):
    if (test["hour"][i] >= 23 or test["hour"][i] <= 5):
        ary.append(1)
    else:
        ary.append(0)
test["is_night"] = ary

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Data Processing - Scaling the Variables</span></h1><br>

In [ ]:
import sklearn
# Concat train and test
all = pd.concat([train,test],ignore_index=True)

# Scaling
month = all['month']
month = np.array(month)
all['month'] = sklearn.preprocessing.minmax_scale(month[:])
dayofweek = all['dayofweek']
dayofweek = np.array(dayofweek)
all['dayofweek'] = sklearn.preprocessing.minmax_scale(dayofweek[:])
hour = all['hour']
hour = np.array(hour)
all['hour'] = sklearn.preprocessing.minmax_scale(hour[:])

deg_C = all['deg_C']
deg_C = np.array(deg_C)
all['deg_C_2'] = sklearn.preprocessing.minmax_scale(deg_C[:])
relative_humidity = all['relative_humidity']
relative_humidity = np.array(relative_humidity)
all['relative_humidity_2'] = sklearn.preprocessing.minmax_scale(relative_humidity[:])
absolute_humidity = all['absolute_humidity']
absolute_humidity = np.array(absolute_humidity)
all['absolute_humidity_2'] = sklearn.preprocessing.minmax_scale(absolute_humidity[:])
sensor_1 = all['sensor_1']
sensor_1 = np.array(sensor_1)
all['sensor_1_2'] = sklearn.preprocessing.minmax_scale(sensor_1[:])
sensor_2 = all['sensor_2']
sensor_2 = np.array(sensor_2)
all['sensor_2_2'] = sklearn.preprocessing.minmax_scale(sensor_2[:])
sensor_3 = all['sensor_3']
sensor_3 = np.array(sensor_3)
all['sensor_3_2'] = sklearn.preprocessing.minmax_scale(sensor_3[:])
sensor_4 = all['sensor_4']
sensor_4 = np.array(sensor_4)
all['sensor_4_2'] = sklearn.preprocessing.minmax_scale(sensor_4[:])
sensor_5 = all['sensor_5']
sensor_5 = np.array(sensor_5)
all['sensor_5_2'] = sklearn.preprocessing.minmax_scale(sensor_5[:])

# Split all for train and test
train_scale = all.iloc[train.index[0]:train.index[-1]+1].drop(columns=["deg_C", "relative_humidity", "absolute_humidity", "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"])
test_scale = all.iloc[train.index[-1]+1:].drop(columns=["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides", "deg_C", "relative_humidity", "absolute_humidity", "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"])
test_scale = pd.DataFrame.reset_index(test_scale).drop(columns=["index"])

In [ ]:
train.head()

In [ ]:
test.head()

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Check the Correlation</span></h1><br>

In [ ]:
corr = train_scale.corr().round(2)

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(corr, vmin=-1, vmax=1, center=0, square=False, annot=True, cmap='coolwarm')
plt.show()

We can notice that the sensor variables seems to have a slight positive coorelation with the target variables.
Time based varibles like summer,weekend and night have slight negative correlation.

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Model Prediction</span></h1><br>

In [ ]:
## drop unnecessary columns
columns_2 = test_scale.columns.drop(['date_time', 'date_time_1', 'month'])#, 'dayofweek', 'hour'

In [ ]:
X = train_scale[columns_2].values
X_test = test_scale[columns_2].values
value_2 = train_scale['target_benzene'].values.reshape(-1,1)

In [ ]:
X_train_2, X_test_2, t_train_2, t_test_2 = train_test_split(X, value_2,test_size=0.3, random_state=0)
lgb_train_2 = lgb.Dataset(X_train_2, t_train_2)
lgb_eval_2 = lgb.Dataset(X_test_2, t_test_2, reference=lgb_train_2)
params_2 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'l2'},
        'learning_rate': 0.002,
        'num_leaves': 16,
        'num_iterations': 20000,
        'verbosity': -1
}
model_2 = lgb.train(
    params_2,
    train_set=lgb_train_2,
    valid_sets=lgb_eval_2,
    early_stopping_rounds=100,
    verbose_eval=100
)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Prediction and Evaluation</span></h1><br>

In [ ]:
pred_2 = model_2.predict(X_test_2)
msle_2 = mean_squared_log_error(t_test_2, pred_2) 
rmsle_2 = np.sqrt(msle_2) 
print('RMSLE_2 : {}'.format(rmsle_2))
r2_2 = r2_score(t_test_2,pred_2)
print('R2_2    : {}'.format(r2_2))

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Predicting target_carbon_monoxide from target_benzene</span></h1><br>

In [ ]:
sample_submission['target_benzene'] = model_2.predict(X_test)

test_scale['target_benzene'] = model_2.predict(X_test)

# Concat train and test
all = pd.concat([train_scale,test_scale],ignore_index=True)
target_benzene = all['target_benzene']
target_benzene = np.array(target_benzene)
all['target_benzene'] = sklearn.preprocessing.minmax_scale(target_benzene[:])

# Split all for train and test
train_scale = all.iloc[train.index[0]:train.index[-1]+1]
test_scale = all.iloc[train.index[-1]+1:].drop(columns=["target_carbon_monoxide", "target_nitrogen_oxides"])

columns_1 = test_scale.columns.drop(['date_time', 'date_time_1', 'month'])#, 'dayofweek', 'hour'
columns_1

In [ ]:
X = train_scale[columns_1].values
X_test = test_scale[columns_1].values
value_1 = train_scale['target_carbon_monoxide'].values.reshape(-1,1)

In [ ]:
X_train_1, X_test_1, t_train_1, t_test_1 = train_test_split(X, value_1,test_size=0.3, random_state=0)

lgb_train_1 = lgb.Dataset(X_train_1, t_train_1)
lgb_eval_1 = lgb.Dataset(X_test_1, t_test_1, reference=lgb_train_1)

params_1 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',# binary、multiclass
        'metric': {'l2'},
        'learning_rate': 0.01,
        'num_leaves': 8,
        'num_iterations': 20000,
        'verbosity': -1
}

model_1 = lgb.train(
    params_1,
    train_set=lgb_train_1,
    valid_sets=lgb_eval_1,
    early_stopping_rounds=100,
    verbose_eval=100
)

# Verification

pred_1 = model_1.predict(X_test_1)

msle_1 = mean_squared_log_error(t_test_1, pred_1)
rmsle_1 = np.sqrt(msle_1)
print('RMSLE_1 : {}'.format(rmsle_1))

r2_1 = r2_score(t_test_1,pred_1)
print('R2_1    : {}'.format(r2_1))


<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Predicting target_nitrogen_oxides from target_carbon_monoxide</span></h1><br>

In [ ]:
sample_submission['target_carbon_monoxide'] = model_1.predict(X_test)

test_scale['target_carbon_monoxide'] = model_1.predict(X_test)

# Concat train and test
all = pd.concat([train_scale,test_scale],ignore_index=True)
target_carbon_monoxide = all['target_carbon_monoxide']
target_carbon_monoxide = np.array(target_carbon_monoxide)
all['target_carbon_monoxide'] = sklearn.preprocessing.minmax_scale(target_carbon_monoxide[:])

# Split all for train and test
train_scale = all.iloc[train.index[0]:train.index[-1]+1]
test_scale = all.iloc[train.index[-1]+1:].drop(columns=["target_nitrogen_oxides"])

columns_3 = test_scale.columns.drop(['date_time', 'date_time_1', 'month'])

In [ ]:
X = train_scale[columns_3].values
X_test = test_scale[columns_3].values
value_3 = train_scale['target_nitrogen_oxides'].values.reshape(-1,1)

In [ ]:
X_train_3, X_test_3, t_train_3, t_test_3 = train_test_split(X, value_3,test_size=0.3, random_state=0)

lgb_train_3 = lgb.Dataset(X_train_3, t_train_3)
lgb_eval_3 = lgb.Dataset(X_test_3, t_test_3, reference=lgb_train_3)

params_3 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'l2'},
        'learning_rate': 0.0005,
        'num_leaves': 92,
        'num_iterations': 20000,
        'verbosity': -1
}

model_3 = lgb.train(
    params_3,
    train_set=lgb_train_3,
    valid_sets=lgb_eval_3,
    early_stopping_rounds=100,
    verbose_eval=100
)


# Verification

pred_3 = model_3.predict(X_test_3)

msle_3 = mean_squared_log_error(t_test_3, pred_3)
rmsle_3 = np.sqrt(msle_3)
print('RMSLE_3 : {}'.format(rmsle_3))

r2_3 = r2_score(t_test_3,pred_3)
print('R2_3    : {}'.format(r2_3))

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Predicting target_nitrogen_oxides</span></h1><br>

In [ ]:
sample_submission['target_nitrogen_oxides'] = model_3.predict(X_test)

sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)